Extraction and quantification of market sentiment from textual data such as news headlines or social media posts related to stock tickers. It fetches recent text data using AlphaVantage Global News API, preprocesses and cleans the text for analysis, and then applies the VADER sentiment analyser to assign sentiment scores (positive, negaitve, neutral and compound) to each piece of text. 
Scores are then aggregated over chosen time intervals to create a time-aligned sentiment dataset that can be merged with market price data for further modeling and visualisation. 

In [17]:
# import libraries 
import requests 
import pandas as pd 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import yfinance as yf 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [3]:
def fetch_alphavantage_news_df(api_url: str) -> pd.DataFrame: 
    """
    Fetches news data from Alphavantage Global News API URL and returns
    a cleaned pandas DataFrame with parsed dates.
    
    Parameters:
        api_url (str): Fully constructed API URL with key and parameters.
    
    Returns:
        pd.DataFrame: DataFrame with news articles.
    """
    response = requests.get(api_url)
    data = response.json()

    # Extract the "feed" list from the JSON response 
    news_list = data.get("feed", [])
    
    # Convert list of news dictionaries into a DataFrame
    df = pd.DataFrame(news_list)

    # Parse the published dates into datetime objects 
    df["time_published"] = pd.to_datetime(df["time_published"], errors = "coerce")

    return df

In [4]:
url = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date= &tickers=AAPL&apikey=1D1C3X346D6ATHPG"

In [ ]:
# Dynamically create the url in order to specify the ticker, date, and apikey

def build_alphavantage_news_url(ticker, date, apikey):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date={date}&tickers={ticker}&apikey={apikey}"
    return url

In [ ]:
# Change this to dynamically fetch a different url 

news_url = build_alphavantage_news_url("AAPL", "2025-07-25","1D1C3X346D6ATHPG")

In [6]:
news_df = fetch_alphavantage_news_df(news_url)

In [7]:
# Visualise the dataframe 
print(news_df.columns.tolist())
print(news_df.head())

['title', 'url', 'time_published', 'authors', 'summary', 'banner_image', 'source', 'category_within_source', 'source_domain', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'ticker_sentiment']
                                               title  \
0                Why Nvidia Stock Popped 13% in July   
1  Why Is Rivian Automotive Gaining Monday? - Riv...   
2  Why Is Snowflake Stock Soaring Monday? - Snowf...   
3  Richard Bernstein Dumps Apple Stock in the Sec...   
4  Nvidia And AMD's Extraordinary Deal For China ...   

                                                 url      time_published  \
0  https://www.fool.com/investing/2025/08/11/why-... 2025-08-11 21:25:00   
1  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-11 17:44:44   
2  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-11 16:35:53   
3  https://www.fool.com/coverage/filings/2025/08/... 2025-08-11 15:34:47   
4  https://www.benzinga.com/general/market-summar... 2025-08-11 15:29:35 

In [8]:
# Apply VADER sentiment analysis on the news headlines using the "title" and "summary" columns

sia = SentimentIntensityAnalyzer()
def get_sentiment_scores(text): 
    if isinstance(text, str):
        return sia.polarity_scores(text)
    else:
        return {'neg': None, 'neu': None, 'pos': None, 'compound': None}


In [9]:
# Create sentiment score columns 
news_df[["neg", "neu", "pos", "compound"]] = news_df["title"].apply(get_sentiment_scores).apply(pd.Series)

In [10]:
news_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,neg,neu,pos,compound
0,Why Nvidia Stock Popped 13% in July,https://www.fool.com/investing/2025/08/11/why-...,2025-08-11 21:25:00,[Danny Vena],It was a busy month for the artificial intelli...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.182269,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1065...",0.000,1.000,0.000,0.0000
1,Why Is Rivian Automotive Gaining Monday? - Riv...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-11 17:44:44,[Anusuya Lahiri],Rivian plans 2025.26 software update with majo...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185579,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.2263...",0.000,0.797,0.203,0.4215
2,Why Is Snowflake Stock Soaring Monday? - Snowf...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-11 16:35:53,[Anusuya Lahiri],Snowflake gains as AI data cloud tools ride $2...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.270984,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.2833...",0.000,1.000,0.000,0.0000
3,Richard Bernstein Dumps Apple Stock in the Sec...,https://www.fool.com/coverage/filings/2025/08/...,2025-08-11 15:34:47,[Lee Samaha],"On Aug. 6, 2025, Richard Bernstein Advisors LL...",https://cdn.content.foolcdn.com/images/1umn9qe...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.225949,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.7223...",0.252,0.748,0.000,-0.4019
4,Nvidia And AMD's Extraordinary Deal For China ...,https://www.benzinga.com/general/market-summar...,2025-08-11 15:29:35,[The Arora Report],"To gain an edge, this is what you need to know...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.189893,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0957...",0.127,0.873,0.000,-0.1531
5,Tredje AP-fonden Loads Up On 1.2M Alphabet ( ...,https://www.fool.com/coverage/filings/2025/08/...,2025-08-11 15:03:00,[Anders Bylund],"On Aug. 5, 2025, Tredje AP-fonden disclosed a ...",https://cdn.content.foolcdn.com/images/1umn9qe...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.248431,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0603...",0.000,0.820,0.180,0.2960
6,Analyzing Apple In Comparison To Competitors I...,https://www.benzinga.com/insights/news/25/08/4...,2025-08-11 15:00:53,[Benzinga Insights],In the dynamic and fiercely competitive busine...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.217761,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.5456...",0.000,1.000,0.000,0.0000
7,Apple: Navigating Tariffs while Services Drive...,https://www.zacks.com/commentary/2697645/apple...,2025-08-11 14:58:00,[Andrew Rocco],Apple is one of the steadiest growers on Wall ...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.231259,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.9144...",0.000,0.698,0.302,0.3818
8,Vanguard's VOO Becomes First ETF to Cross $700B,https://www.zacks.com/stock/news/2697610/vangu...,2025-08-11 14:00:00,[Zacks Investment Research],VOO becomes the first ETF to surpass $700B in ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.294638,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.2802...",0.00

In [11]:
news_df.isnull().sum()

title                      0
url                        0
time_published             0
authors                    0
summary                    0
banner_image               0
source                     0
category_within_source     0
source_domain              0
topics                     0
overall_sentiment_score    0
overall_sentiment_label    0
ticker_sentiment           0
neg                        0
neu                        0
pos                        0
compound                   0
dtype: int64

In [12]:
### Exploration of Sentiment Data 

## Structural inspection 
# Check the first few rows 
print(news_df.head())

# Check the column names 
print("\nColumn names: ")
print(news_df.columns.tolist())

# Check data types and missing values 
print("\nDataFrame info:")
print(news_df.info())

# Check for missing values 
print("\nMissing values per column:")
print(news_df.isnull().sum())

# Quick statistics for numerical columns: 
print("\nSummary statistics:")
print(news_df.describe())

                                               title  \
0                Why Nvidia Stock Popped 13% in July   
1  Why Is Rivian Automotive Gaining Monday? - Riv...   
2  Why Is Snowflake Stock Soaring Monday? - Snowf...   
3  Richard Bernstein Dumps Apple Stock in the Sec...   
4  Nvidia And AMD's Extraordinary Deal For China ...   

                                                 url      time_published  \
0  https://www.fool.com/investing/2025/08/11/why-... 2025-08-11 21:25:00   
1  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-11 17:44:44   
2  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-11 16:35:53   
3  https://www.fool.com/coverage/filings/2025/08/... 2025-08-11 15:34:47   
4  https://www.benzinga.com/general/market-summar... 2025-08-11 15:29:35   

              authors                                            summary  \
0        [Danny Vena]  It was a busy month for the artificial intelli...   
1    [Anusuya Lahiri]  Rivian plans 2025.26 software u

All the columns have the expected names and types 
There are 0 missing values 
The time_published is a datetime object which is to be expected

In [13]:
### Extract relevant ticker-level information from the ticker_sentiment column 

# Extract "ticker" and "relevance_score" into separate columns 
news_df_exploded = news_df.explode("ticker_sentiment").reset_index(drop = True)
ticker_sentiment_expanded = pd.json_normalize(news_df_exploded["ticker_sentiment"])
news_df_final = pd.concat([news_df_exploded.drop(columns = ["ticker_sentiment"]), ticker_sentiment_expanded], axis = 1)
news_df_final.drop(columns = "category_within_source", inplace = True)
news_df_final

,title,url,time_published,authors,summary,banner_image,source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,neg,neu,pos,compound,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
0,Why Nvidia Stock Popped 13% in July,https://www.fool.com/investing/2025/08/11/why-...,2025-08-11 21:25:00,[Danny Vena],It was a busy month for the artificial intelli...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.182269,Somewhat-Bullish,0.000,1.000,0.000,0.0000,MSFT,0.106541,0.002205,Neutral
1,Why Nvidia Stock Popped 13% in July,https://www.fool.com/investing/2025/08/11/why-...,2025-08-11 21:25:00,[Danny Vena],It was a busy month for the artificial intelli...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.182269,Somewhat-Bullish,0.000,1.000,0.000,0.0000,NVDA,0.859307,0.230754,Somewhat-Bullish
2,Why Nvidia Stock Popped 13% in July,https://www.fool.com/investing/2025/08/11/why-...,2025-08-11 21:25:00,[Danny Vena],It was a busy month for the artificial intelli...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.182269,Somewhat-Bullish,0.000,1.000,0.000,0.0000,SPGI,0.106541,0.299965,Somewhat-Bullish
3,Why Nvidia Stock Popped 13% in July,https://www.fool.com/investing/2025/08/11/why-...,2025-08-11 21:25:00,[Danny Vena],It was a busy month for the artificial intelli...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.182269,Somewhat-Bullish,0.000,1.000,0.000,0.0000,AAPL,0.106541,0.002205,Neutral
4,Why Is Rivian Automotive Gaining Monday? - Riv...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-11 17:44:44,[Anusuya Lahiri],Rivian plans 2025.26 software update with majo...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185579,Somewhat-Bullish,0.000,0.797,0.203,0.4215,GOOG,0.226301,0.138334,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,"Nvidia, Broadcom Lead Semiconductor Surge As A...",https://www.benzinga.com/markets/tech/25/08/47...,2025-08-08 15:14:41,[Anusuya Lahiri],"Semiconductor stocks rose as AMD, Super Micro,...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.234005,Somewhat-Bullish,0.047,0.791,0.161,0.5574,MU,0.13347,0.219665,Somewhat-Bullish
268,"Nvidia, Broadcom Lead Semiconductor Surge As A...",https://www.benzinga.com/markets/tech/25/08/47...,2025-08-08 15:14:41,[Anusuya Lahiri],"Semiconductor stocks rose as AMD, Super Micro,...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.234005,Somewhat-Bullish,0.047,0.791,0.161,0.5574,ARM,0.13347,0.219665,Somewhat-Bullish
269,"Nvidia, Broadcom Lead Semiconductor Surge As A...",https://www.benzinga.com/markets/tech/25/08/47...,2025-08-08 15:14:41,[Anusuya Lahiri],"Semiconductor stocks rose as AMD, Super Micro,...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.234005,Somewhat-Bullish,0.047,0.791,0.161,0.5574,MRVL,0.13347,0.219665,Somewhat-Bullish
270,Weekly Option Windfall: Trading AAPL Options A...,https://www.zacks.com/commentary/2684377/weekl...,2025-08-08 15:09:00,[Bryan Hayes],The tech giant is slated to release the newest...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.294286,Somewhat-Bullish,0.000,1.000,0.000,0.0000,AAPL,0.71032,0.51126,Bullish


In [ ]:
### Aggregate sentiment data at the level appropriate such that it matches the market data- daily per ticker 

# Sort the dataframe by ticker 
news_df_sorted = news_df_final.sort_values(by = "ticker")

# Convert time_published to date only 
news_df_sorted["time_published"] = pd.to_datetime(news_df_sorted["time_published"], errors = "coerce")
news_df_sorted["date"] = news_df_sorted["time_published"].dt.day

# Sort the values by ticker and date 
news_df_sorted.sort_values(by = ["ticker", "date"], inplace = True)

## Combine the multiple sentiment scores into a single summary value per group 

# New column with weighted sentiment per article 
news_df_sorted["ticker_sentiment_score"] = pd.to_numeric(news_df_sorted["ticker_sentiment_score"], errors = "coerce")
news_df_sorted["relevance_score"] = pd.to_numeric(news_df_sorted["relevance_score"], errors = "coerce")
news_df_sorted["weighted_sentiment_score"] = news_df_sorted["ticker_sentiment_score"] * news_df_sorted["relevance_score"]

# Group by ticket and date and then aggregate to sum the weighted sentiments, relevance scores and compute the weighted average sentiment by dividng these sums for each news article 
grouped = news_df_sorted.groupby(["ticker", "date"]).agg(
    total_weighted_sentiment = ("weighted_sentiment_score", "sum"),
    total_relevance = ("relevance_score", "sum")
).reset_index()  # optional, to turn MultiIndex into columns

grouped["weighted_avg_sentiment"] = grouped["total_weighted_sentiment"] / grouped["total_relevance"]

grouped

,ticker,date,total_weighted_sentiment,total_relevance,weighted_avg_sentiment
0,AAPL,8,1.868519,4.589519,0.407127
1,AAPL,9,0.923587,4.744314,0.194672
2,AAPL,10,0.727302,2.276524,0.319479
3,AAPL,11,1.734102,7.389001,0.234687
4,ABNB,10,0.000000,0.074576,0.000000
...,...,...,...,...,...
137,WBD,9,-0.002638,0.068698,-0.038393
138,WBD,11,-0.002638,0.068698,-0.038393
139,WLDS,10,0.003150,0.050894,0.061897
140,XIACY,11,0.008716,0.045830,0.190192


In [77]:
### Coordinate the sentiment data and the market data 

# Determine date range from sentiment data to define the market data window
start_date = (news_df_final["time_published"]).dt.date.min()
end_date = (news_df_final["time_published"]).dt.date.max()

# Extract unique tickers from news_df_final["ticker"]
unique_tickers = news_df_final["ticker"].unique()

print(start_date, end_date)

2025-08-08 2025-08-11


In [61]:
unique_tickers

array(['MSFT', 'NVDA', 'SPGI', 'AAPL', 'GOOG', 'LCID', 'RIVN', 'TSLA',
       'META', 'BAC', 'SNOW', 'MS', 'IVZ', 'AMZN', 'SSNLF', 'ON', 'TSM',
       'MU', 'ASCCF', 'FOREX:AMD', 'BABA', 'TCTZF', 'FOREX:USD', 'NFLX',
       'SHOP', 'AVGO', 'WBD', 'TRI', 'C', 'XIACY', 'APP', 'BNXYF', 'CDNS',
       'INTC', 'MELI', 'UPST', 'EFX', 'TRU', 'SVNDF', 'UNP', 'BRK-A',
       'KHC', 'NSC', 'UBER', 'TTD', 'BSQKZ', 'F', 'GM', 'SMCI', 'DDOG',
       'ABNB', 'DASH', 'DJT', 'HYMLF', 'PLTR', 'AMAT', 'MP', 'TXN',
       'WLDS', 'AMKR', 'GLW', 'GFS', 'LLY', 'RUN', 'GS', 'JOBY', 'OXY',
       'MA', 'V', 'NU', 'XOM', 'AXP', 'DVA', 'INTU', 'ORCL', 'NOW',
       'PANW', 'CCZ', 'ANET', 'AMD', 'DELL', 'PBR', 'FLUT', 'MNST',
       'AKAM', 'LYV', 'TTWO', 'BACHY', 'DIS', 'QCOM', 'ARM', 'MRVL'],
      dtype=object)

In [84]:
# Download data from yfinance dataset 

def fetch_and_process_daily_data(ticker, start_date, end_date):
    try:
        df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=False)
        print(f"Downloaded {ticker}: columns={df.columns.tolist()}, rows={len(df)}")  # debug
        
        if df.empty or len(df) < 2:
            print(f"Not enough data to calculate returns for ticker {ticker}")
            return None
        
        if "Close" not in df.columns:
            print(f"'Close' column missing for ticker {ticker}")
            return None
        
        df["log_return"] = np.log(df["Close"] / df["Close"].shift(1))
        
        if df.empty:
            print(f"After computing log_return, no data left for ticker {ticker}")
            return None
        
        return df

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None




In [85]:
# Create data frame containing all fetched market data given a ticker and start and end dates
market_data = []

for ticker in unique_tickers:
    df_ticker = fetch_and_process_daily_data(ticker, "2025-07-08", "2025-08-11")
    if df_ticker is not None:
        df_ticker["Ticker"] = ticker
        market_data.append(df_ticker)
    else:
        print(f"Warning: No data for ticker {ticker}") 

if market_data:
    final_df = pd.concat(market_data)
else:
    print("No market data collected for any ticker.")
    final_df = pd.DataFrame()  # empty dataframe fallback


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded MSFT: columns=[('Adj Close', 'MSFT'), ('Close', 'MSFT'), ('High', 'MSFT'), ('Low', 'MSFT'), ('Open', 'MSFT'), ('Volume', 'MSFT')], rows=24
Downloaded NVDA: columns=[('Adj Close', 'NVDA'), ('Close', 'NVDA'), ('High', 'NVDA'), ('Low', 'NVDA'), ('Open', 'NVDA'), ('Volume', 'NVDA')], rows=24
Downloaded SPGI: columns=[('Adj Close', 'SPGI'), ('Close', 'SPGI'), ('High', 'SPGI'), ('Low', 'SPGI'), ('Open', 'SPGI'), ('Volume', 'SPGI')], rows=24
Downloaded AAPL: columns=[('Adj Close', 'AAPL'), ('Close', 'AAPL'), ('High', 'AAPL'), ('Low', 'AAPL'), ('Open', 'AAPL'), ('Volume', 'AAPL')], rows=24
Downloaded GOOG: columns=[('Adj Close', 'GOOG'), ('Close', 'GOOG'), ('High', 'GOOG'), ('Low', 'GOOG'), ('Open', 'GOOG'), ('Volume', 'GOOG')], rows=24
Downloaded LCID: columns=[('Adj Close', 'LCID'), ('Close', 'LCID'), ('High', 'LCID'), ('Low', 'LCID'), ('Open', 'LCID'), ('Volume', 'LCID')], rows=24
Downloaded RIVN: columns=[('Adj Close', 'RIVN'), ('Close', 'RIVN'), ('High', 'RIVN'), ('Low', 'RIVN'

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded SSNLF: columns=[('Adj Close', 'SSNLF'), ('Close', 'SSNLF'), ('High', 'SSNLF'), ('Low', 'SSNLF'), ('Open', 'SSNLF'), ('Volume', 'SSNLF')], rows=24
Downloaded ON: columns=[('Adj Close', 'ON'), ('Close', 'ON'), ('High', 'ON'), ('Low', 'ON'), ('Open', 'ON'), ('Volume', 'ON')], rows=24


[*********************100%***********************]  1 of 1 completed


...

Downloaded QCOM: columns=[('Adj Close', 'QCOM'), ('Close', 'QCOM'), ('High', 'QCOM'), ('Low', 'QCOM'), ('Open', 'QCOM'), ('Volume', 'QCOM')], rows=24


[*********************100%***********************]  1 of 1 completed


Downloaded ARM: columns=[('Adj Close', 'ARM'), ('Close', 'ARM'), ('High', 'ARM'), ('Low', 'ARM'), ('Open', 'ARM'), ('Volume', 'ARM')], rows=24


[*********************100%***********************]  1 of 1 completed

Downloaded MRVL: columns=[('Adj Close', 'MRVL'), ('Close', 'MRVL'), ('High', 'MRVL'), ('Low', 'MRVL'), ('Open', 'MRVL'), ('Volume', 'MRVL')], rows=24


In [87]:
final_df

Price,Adj Close,Close,High,Low,Open,Volume,log_return,Ticker,Adj Close,Close,...,High,Low,Open,Volume,Adj Close,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT,,,NVDA,NVDA,...,ARM,ARM,ARM,ARM,MRVL,MRVL,MRVL,MRVL,MRVL,MRVL
Date,,,,,,,,,,,,,,,,,,,,,
2025-07-08,496.619995,496.619995,498.200012,494.109985,497.239990,11846600.0,NaN,MSFT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-09,503.510010,503.510010,506.779999,499.739990,500.299988,18659500.0,0.013778,MSFT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-10,501.480011,501.480011,504.440002,497.750000,503.049988,16492100.0,-0.004040,MSFT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-11,503.320007,503.320007,505.029999,497.799988,498.470001,16459500.0,0.003662,MSFT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-07-14,503.019989,503.019989,503.970001,501.029999,501.519989,12058800.0,-0.000596,MSFT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04,NaN,NaN,NaN,NaN,NaN,NaN,0.027555,MRVL,NaN,NaN,...,NaN,NaN,NaN,NaN,76.529999,76.529999,76.889999,74.820000,75.699997,11882900.0
2025-08-05,NaN,NaN,NaN,NaN,NaN,NaN,0.001306,MRVL,NaN,NaN,...,NaN,NaN,NaN,NaN,76.629997,76.629997,78.000000,74.959999,76.910004,11920800.0


In [86]:
final_df.columns

MultiIndex([( 'Adj Close', 'MSFT'),
            (     'Close', 'MSFT'),
            (      'High', 'MSFT'),
            (       'Low', 'MSFT'),
            (      'Open', 'MSFT'),
            (    'Volume', 'MSFT'),
            ('log_return',     ''),
            (    'Ticker',     ''),
            ( 'Adj Close', 'NVDA'),
            (     'Close', 'NVDA'),
            ...
            (      'High',  'ARM'),
            (       'Low',  'ARM'),
            (      'Open',  'ARM'),
            (    'Volume',  'ARM'),
            ( 'Adj Close', 'MRVL'),
            (     'Close', 'MRVL'),
            (      'High', 'MRVL'),
            (       'Low', 'MRVL'),
            (      'Open', 'MRVL'),
            (    'Volume', 'MRVL')],
           names=['Price', 'Ticker'], length=542)

In [63]:
final_df

Price,Date,Close,High,Low,Open,Volume,log_return,Ticker,Adj Close


In [53]:
### Process sentiment data 

# Group by date & ticker to get daily average sentiment using weighted sentiment score
sentiment_daily = (
    news_df_sorted
    .groupby(["date", "ticker"])["weighted_sentiment_score"]
    .mean()
    .reset_index()
)

### Process market data
final_df["date"] = pd.to_datetime(final_df["date"])

### Merge market and sentiment data 
merged_df = pd.merge(final_df, sentiment_daily, on=["date", "ticker"], how="left")

KeyError: 'date'

In [49]:
merged_df

,title,url,time_published,authors,summary,banner_image,source,source_domain,topics,overall_sentiment_score,...,neu,pos,compound,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label,date,weighted_sentiment_score_x,weighted_sentiment_score_y
0,Market Analysis: Apple And Competitors In Tech...,https://www.benzinga.com/insights/news/25/08/4...,2025-08-08 15:00:46,[Benzinga Insights],In today's rapidly changing and fiercely compe...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.269095,...,1.000,0.000,0.0000,AAPL,0.495015,0.324811,Somewhat-Bullish,8,0.160786,0.169865
1,Weekly Option Windfall: Trading AAPL Options A...,https://www.zacks.com/commentary/2684377/weekl...,2025-08-08 15:09:00,[Bryan Hayes],The tech giant is slated to release the newest...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.294286,...,1.000,0.000,0.0000,AAPL,0.710320,0.511260,Bullish,8,0.363158,0.169865
2,Why Shares of Apple Are Surging Today,https://www.fool.com/investing/2025/08/08/why-...,2025-08-08 18:47:51,[Bram Berkowitz],Apple's tariff troubles seem to be winding down.,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.178532,...,0.732,0.268,0.2960,AAPL,0.512093,0.241714,Somewhat-Bullish,8,0.123780,0.169865
3,"Nvidia, Broadcom Lead Semiconductor Surge As A...",https://www.benzinga.com/markets/tech/25/08/47...,2025-08-08 15:14:41,[Anusuya Lahiri],"Semiconductor stocks rose as AMD, Super Micro,...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.234005,...,0.791,0.161,0.5574,AAPL,0.263231,0.343995,Somewhat-Bullish,8,0.090550,0.169865
4,Parks & Streaming Drive Disney's Q3 Results: T...,https://www.zacks.com/stock/news/2684483/parks...,2025-08-08 15:34:00,[Vasundhara Sawalka],DIS' Q3 beat shows record parks revenues and s...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,www.zacks.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.403366,...,1.000,0.000,0.0000,AAPL,0.077845,0.157770,Somewhat-Bullish,8,0.012282,0.169865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,AAPL NEWS: Did Apple Inc. Mislead Investors? C...,https://www.benzinga.com/pressreleases/25/08/g...,2025-08-09 11:36:00,[Globe Newswire],"NEW YORK, Aug. 09, 2025 ( GLOBE NEWSWIRE ) -- ...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.145500,...,1.000,0.000,0.0000,WBD,0.068698,-0.038393,Neutral,9,-0.002638,-0.002638
268,AAPL COURT NOTICE: Apple Inc. Investors may ha...,https://www.benzinga.com/pressreleases/25/08/g...,2025-08-11 12:18:00,[Globe Newswire],"NEW YORK, Aug. 11, 2025 ( GLOBE NEWSWIRE ) -- ...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.145500,...,0.705,0.000,-0.8834,WBD,0.068698,-0.038393,Neutral,11,-0.002638,-0.002638
269,Apple's $600 Billion U.S. Investment Could Res...,https://www.fool.com/investing/2025/08/10/appl...,2025-08-10 07:45:00,[Patrick Sanders],Investors are hoping that Wednesday's announce...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.257684,...,1.000,0.000,0.0000,WLDS,0.050894,0.061897,Neutral,10,0.003150,0.003150
270,Apple's Silence on AI Glasses and Deals: Strat...,https://www.fool.com/investing/2025/08/11/appl...,2025-08-11 08:44:00,[Keith Speights],Apple's management isn't saying much about two...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.213084,...,0.844,0.000,-0.3400,XIACY,0.045830,0.190192,Somewhat-Bullish,11,0.00